In [ ]:
import cv2
import numpy as np
import random as rng
from math import sqrt

In [ ]:
def getIntersectionPoints(coeff_1: tuple, coeff_2: tuple):
  a1, b1, c1 = coeff_1
  a2, b2, c2 = coeff_2

  x = 0 
  y = 0

  det = a1 * b2 - a2 * b1

  x_num = b1 * c2 - b2 * c1

  y_num = c1 * a2 - c2 * a1

  # lines are approximately parallel 
  if det > -0.5 and det < 0.5:
    return None
  

  if det != 0:
    x = x_num / det
    y = y_num / det
    return (x, y)
  
  return None

In [ ]:
def distanceBetweenPoints(p1: tuple, p2: tuple):
  x1, y1 = p1
  x2, y2, = p2
  distance = sqrt((x2 - x1)**2 + (y2 - y1)**2)
  return distance

In [ ]:
def areSimilarCorners(c1: tuple, c2: tuple): 
  return True if distanceBetweenPoints(c1, c2) < 50 else False

In [ ]:
def angle(x1, y1, x2, y2): 
    # Use dotproduct to find angle between vectors 
    # # This always returns an angle between 0, pi 
    numer = (x1 * x2 + y1 * y2) 
    denom = np.sqrt((x1 ** 2 + y1 ** 2) * (x2 ** 2 + y2 ** 2)) 
    
    return np.arccos(numer / denom) * 180 / np.pi 

In [ ]:
def getInnerBoundingBox(width, height, scale):
  innerWidth = width * scale
  innerHeight = height * scale

  y0 = (height - innerHeight)/2
  x0 = (width - innerWidth)/2

  tl = (x0, y0)
  tr = (x0 + innerWidth,  y0)
  br = (x0 + innerWidth, y0 + innerHeight)
  bl = (x0, y0 + innerHeight)
  
  return [tl, tr, br, bl]

In [ ]:
def withinBoundingCornerProximity(corner, boundingCorner, width, height): 
  xc, yc = corner
  distance = distanceBetweenPoints(corner, boundingCorner)
  print("DISTANCE BETWEEN CORNERS: ", distance)
  if distance < 500 and xc > 0 and xc < width and yc > 0 and yc < height:
    return True
  return False
    


In [ ]:
def refineCorners(allCorners, boundingCorners):
  distinctCorners = []
  # refine corners based on their distance
  for i in range(len(allCorners)): 
    # check with all other corners
    similar_corner = False
    for j in range(len(allCorners)): 
      # if we have checked all corners upto this one then break
      if i == j:
        break
      # if corner is similar to any of the previous corners then we do not add this corner
      if areSimilarCorners(allCorners[i], allCorners[j]): 
        similar_corner = True
        break
    if not similar_corner:
      distinctCorners.append(allCorners[i])

  corners = []  
  if len(distinctCorners) == 4:
    orderedCorners = orderPoints(distinctCorners)

    tl, tr, br, bl = boundingCorners

    boundingWidth = tr[0] - tl[1]
    boundingHeight = br[1] - tr[1]

    for corner, boundingCorner in zip(orderedCorners, boundingCorners):
      print("CORNER", corner)
      print("BOUNDING CORNER:", boundingCorner)
      # if withinBoundingCornerProximity(corner, boundingCorner, boundingWidth, boundingHeight):
      #   corners.append(corner)


    if len(orderedCorners) == 4:
      for i in range(len(orderedCorners)):
        p1 = orderedCorners[i]
        ref = orderedCorners[i - 1]
        p2 = orderedCorners[i - 2]
        
        x1, y1 = p1[0] - ref[0], p1[1] - ref[1]
        x2, y2 = p2[0] - ref[0], p2[1] - ref[1]

        if angle(x1, y1, x2, y2) < 100:
          corners.append(ref) 

  return corners if len(corners) == 4 else None

In [ ]:
def getAllPossibleCorners(line_equations, row_size, col_size): 
  all_corners = []

  # for every combination of coeffs get the intersection points
  for i in range(len(line_equations)):
    for j in range(i, len(line_equations)):
      if(i != j): 
      
        int_point = getIntersectionPoints(line_equations[i], line_equations[j])
        if int_point != None: 
          x, y = int_point
          # coords should be within the image boundaries
          if x > 0 and y > 0 and x < col_size and y < row_size:
            all_corners.append(int_point)
            
  return all_corners

In [ ]:
def orderPoints(points: list):
	# initialize a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	pts = np.array(points)
	print("points", points)
	print("--------------")
	print("points array", pts)
	print("--------------")
	s = pts.sum(axis = 1)

	print("sum", s)
	print("--------------")
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	print("diff", diff)
	print("--------------")
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

print(orderPoints([(10,0), (0,10), (10, 10), (0,0)]))


In [ ]:
def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = orderPoints(pts)
	WIDTH = 480
	HEIGHT = 640

	dst = np.array([
		[0, 0],
		[WIDTH - 1, 0],
		[WIDTH - 1, HEIGHT - 1],
		[0, HEIGHT - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (WIDTH, HEIGHT))
	# return the warped image
	return warped

In [ ]:
def sortContours(elem):
    return cv2.arcLength(elem, closed=True)

In [ ]:
def getConvexHullMask(image):
  # find the contours
  contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

  # get the largest contour by arc length
  contours = sorted(contours, key=sortContours, reverse=True)[:1]

  convex_hull_mask = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

  convex_hull_mask_grayscale = cv2.cvtColor(convex_hull_mask, cv2.COLOR_BGR2GRAY)

  # convex hull object
  hull_list = []
  hull = cv2.convexHull(contours[-1], True)
  hull_list.append(hull)

  cv2.drawContours(convex_hull_mask_grayscale, hull_list, -1, (255,0,0), 2, 8)

  return convex_hull_mask_grayscale

In [ ]:
def getLineCoefficients(p1: tuple, p2: tuple):
  x1, y1 = p1
  x2, y2 = p2

  a = y1 - y2
  b = x2 - x1
  c = x1*y2 - x2*y1

  return (a,b,c)

In [ ]:
def getLineEquations(lines): 
  line_equations = []
  for line in lines:
        l = line[0]
        line_equations.append(getLineCoefficients((l[0], l[1]), (l[2], l[3])))
  
  return line_equations

In [ ]:
def drawBoundingBox(image, corners, color=(0,0,255), thickness=5):
  tl = corners[0]
  tr = corners[1]
  br = corners[2]
  bl = corners[3]

  cv2.line(image, (int(tl[0]), int(tl[1])), (int(tr[0]), int(tr[1])), color, thickness, cv2.LINE_AA )

  cv2.line(image, (int(tr[0]), int(tr[1])), (int(br[0]), int(br[1])), color, thickness, cv2.LINE_AA )

  cv2.line(image, (int(br[0]), int(br[1])), (int(bl[0]), int(bl[1])), color, thickness, cv2.LINE_AA )

  cv2.line(image, (int(bl[0]), int(bl[1])), (int(tl[0]), int(tl[1])), color, thickness, cv2.LINE_AA )


In [43]:
def scanDocument(src_img): 

  resized_img = cv2.resize(src_img, (480, 640))
  output = resized_img.copy()

  grayscale_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

  cv2.namedWindow("GRAYSCALE", cv2.WINDOW_NORMAL)
  cv2.imshow("GRAYSCALE", grayscale_img)
  cv2.moveWindow("GRAYSCALE", 0, 0)
  cv2.resizeWindow("GRAYSCALE", 300, 400)
  cv2.waitKey(10)

  kernel = np.ones((3,3),np.uint8)
  dilation = cv2.dilate(grayscale_img,kernel,iterations=5)
  blur_img = cv2.GaussianBlur(dilation, (3,3), 0)
  blur_img= cv2.erode(blur_img,kernel,iterations=5)

  cv2.namedWindow("BLUR", cv2.WINDOW_NORMAL)
  cv2.imshow("BLUR", blur_img)
  cv2.moveWindow("BLUR", 0, 400)
  cv2.resizeWindow("BLUR", 300, 400)
  cv2.waitKey(10)

  # thresh_img = cv2.adaptiveThreshold(blur_img,150,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,2)

  thresh_img = cv2.adaptiveThreshold(blur_img,150,cv2.
  ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,2)

  cv2.namedWindow("THRESH", cv2.WINDOW_NORMAL)
  cv2.imshow("THRESH", thresh_img)
  cv2.moveWindow("THRESH", 300, 0)
  cv2.resizeWindow("THRESH", 300, 400)
  cv2.waitKey(10)

  canny_img = cv2.Canny(thresh_img, 50, 100)

  cv2.namedWindow("CANNY", cv2.WINDOW_NORMAL)
  cv2.imshow("CANNY", canny_img)
  cv2.moveWindow("CANNY", 300, 400)
  cv2.resizeWindow("CANNY", 300, 400) # 3/4 aspect ratio of 480x640
  cv2.waitKey(10)

  convex_hull = getConvexHullMask(canny_img)

  cv2.namedWindow("HULL", cv2.WINDOW_NORMAL)
  cv2.imshow("HULL", convex_hull)
  cv2.moveWindow("HULL", 600, 0)
  cv2.resizeWindow("HULL", 300, 400)
  cv2.waitKey(10)


  lines = cv2.HoughLinesP(convex_hull, rho = 1, theta = np.pi / 180, minLineLength=200, maxLineGap=50, threshold=100)

  if lines is not None and len(lines) >= 4:
    line_equations = getLineEquations(lines)

    rows, cols = canny_img.shape

    allCorners = getAllPossibleCorners(line_equations, rows, cols)

    boundingBoxCorners = getInnerBoundingBox(cols, rows, 0.7)

    # drawBoundingBox(output, boundingBoxCorners, (0,255,0), 2)
    
    corners = refineCorners(allCorners, boundingBoxCorners)

    if corners is not None and len(corners) == 4:
      # draw a bounding box on the original image copy
      drawBoundingBox(output, corners)

  cv2.namedWindow("OUTPUT", cv2.WINDOW_NORMAL)
  cv2.imshow("OUTPUT", output)
  cv2.moveWindow("OUTPUT", 600, 400)
  cv2.resizeWindow("OUTPUT", 300, 400)
  cv2.waitKey(10)

In [ ]:
video = cv2.VideoCapture('videos/phone-video.mp4')
 
while (video.isOpened()):
    ret, frame = video.read()
    if frame is not None: 
        scanDocument(frame)
        
video.release()
cv2.destroyAllWindows()